## multithreading

In [1]:
import logging

logging.basicConfig(
    level=logging.INFO,
    datefmt="%H:%M:%S",
    format="%(asctime)s.%(msecs)03d %(funcName)15s:%(lineno)-3d - %(message)s",
)

log = logging.getLogger(__name__)

In [2]:
log.info("hi")

20:23:25.442        <module>:1   - hi


In [4]:
from time import sleep


def get_weather():
    log.info("fetching weather")
    sleep(1)
    log.info("got weather")


def get_stocks():
    log.info("fetching stocks")
    sleep(1)
    log.info("got stocks")

In [5]:
def main():
    log.info("start main")
    get_weather()
    get_stocks()
    log.info("done main")

main()

20:25:16.927            main:2   - start main
20:25:16.929     get_weather:5   - fetching weather
20:25:17.940     get_weather:7   - got weather
20:25:17.941      get_stocks:11  - fetching stocks
20:25:18.946      get_stocks:13  - got stocks
20:25:18.948            main:5   - done main


In [7]:
import threading

def main():
    log.info("start main")
    
    thread_weather = threading.Thread(target=get_weather)
    log.info("created thread")

    thread_weather.start()
    log.info("started thread, will wait")

    thread_weather.join()
    log.info("finished thread")
    
    log.info("done main")

main()

20:28:14.530            main:4   - start main
20:28:14.543            main:7   - created thread
20:28:14.545     get_weather:5   - fetching weather
20:28:14.547            main:10  - started thread, will wait
20:28:15.549     get_weather:7   - got weather
20:28:15.550            main:13  - finished thread
20:28:15.551            main:15  - done main


In [8]:
import threading

def main():
    log.info("start main")
    
    thread_weather = threading.Thread(target=get_weather)
    thread_stocks = threading.Thread(target=get_stocks)
    log.info("created threads")

    thread_weather.start()
    thread_stocks.start()
    log.info("started threads, will wait")

    thread_weather.join()
    thread_stocks.join()
    log.info("finished thread")
    
    log.info("done main")

main()

20:31:29.713            main:4   - start main
20:31:29.719            main:8   - created threads
20:31:29.721     get_weather:5   - fetching weather
20:31:29.723      get_stocks:11  - fetching stocks
20:31:29.724            main:12  - started threads, will wait
20:31:30.726     get_weather:7   - got weather
20:31:30.730      get_stocks:13  - got stocks
20:31:30.731            main:16  - finished thread
20:31:30.732            main:18  - done main


In [9]:
def get_stock_info(name):
    log.info("find stock %s", name)
    sleep(1)
    log.info("got stock %s", name)
    return {"name": name}

In [10]:
def main():
    log.info("start main")
    
    thread_stock_a = threading.Thread(target=get_stock_info, args=("AAA", ))
    thread_stock_b = threading.Thread(target=get_stock_info, args=("BBB", ))
    log.info("created threads")

    thread_stock_a.start()
    thread_stock_b.start()
    log.info("started threads, will wait")

    thread_stock_a.join()
    thread_stock_b.join()
    log.info("finished threads")
    
    log.info("done main")

main()

20:35:18.712            main:2   - start main
20:35:18.713            main:6   - created threads
20:35:18.713  get_stock_info:2   - find stock AAA
20:35:18.714  get_stock_info:2   - find stock BBB
20:35:18.714            main:10  - started threads, will wait
20:35:19.717  get_stock_info:4   - got stock BBB
20:35:19.720  get_stock_info:4   - got stock AAA
20:35:19.722            main:14  - finished threads
20:35:19.723            main:16  - done main


In [13]:
import concurrent.futures

def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(get_stock_info, ["one", "two", "three", "four"])

main()

20:39:17.901  get_stock_info:2   - find stock one
20:39:17.902  get_stock_info:2   - find stock two
20:39:17.903  get_stock_info:2   - find stock three
20:39:18.910  get_stock_info:4   - got stock one
20:39:18.914  get_stock_info:2   - find stock four
20:39:18.911  get_stock_info:4   - got stock two
20:39:18.915  get_stock_info:4   - got stock three
20:39:19.928  get_stock_info:4   - got stock four


In [14]:

def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        results = executor.map(get_stock_info, ["one", "two", "three", "four"])

    for res in results:
        log.info("result: %s", res)

main()

20:40:34.865  get_stock_info:2   - find stock one
20:40:34.866  get_stock_info:2   - find stock two
20:40:34.867  get_stock_info:2   - find stock three
20:40:35.871  get_stock_info:4   - got stock one
20:40:35.872  get_stock_info:2   - find stock four
20:40:35.872  get_stock_info:4   - got stock two
20:40:35.874  get_stock_info:4   - got stock three
20:40:36.874  get_stock_info:4   - got stock four
20:40:36.876            main:6   - result: {'name': 'one'}
20:40:36.881            main:6   - result: {'name': 'two'}
20:40:36.883            main:6   - result: {'name': 'three'}
20:40:36.885            main:6   - result: {'name': 'four'}


In [15]:

def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        future_stock = executor.submit(get_stock_info, "spam")

    res = future_stock.result()
    log.info("result: %s", res)

main()

20:42:00.012  get_stock_info:2   - find stock spam
20:42:01.014  get_stock_info:4   - got stock spam
20:42:01.015            main:6   - result: {'name': 'spam'}


### Global Interpreter Lock

In [16]:
timeout = 42_000_000

def countdown(n):
    log.info("countdown %s", n)
    while n:
        n -= 1
    log.info("done countdown")

In [17]:
countdown(timeout)

20:46:12.929       countdown:4   - countdown 42000000
20:46:13.768       countdown:7   - done countdown


In [18]:
def main():
    log.info("start main")
    
    thread_a = threading.Thread(target=countdown, args=(timeout, ))
    thread_b = threading.Thread(target=countdown, args=(timeout, ))
    log.info("created threads")

    thread_a.start()
    thread_b.start()
    log.info("started threads, will wait")

    thread_a.join()
    thread_b.join()
    log.info("finished threads")
    
    log.info("done main")

main()

20:47:11.617            main:2   - start main
20:47:11.621            main:6   - created threads
20:47:11.622       countdown:4   - countdown 42000000
20:47:11.642       countdown:4   - countdown 42000000
20:47:11.649            main:10  - started threads, will wait
20:47:13.171       countdown:7   - done countdown
20:47:13.229       countdown:7   - done countdown
20:47:13.230            main:14  - finished threads
20:47:13.230            main:16  - done main


In [19]:
%%time

countdown(timeout)

20:48:06.468       countdown:4   - countdown 42000000
20:48:07.280       countdown:7   - done countdown


CPU times: user 794 ms, sys: 6.9 ms, total: 801 ms
Wall time: 812 ms


In [20]:
%%time

main()

20:48:21.098            main:2   - start main
20:48:21.102            main:6   - created threads
20:48:21.103       countdown:4   - countdown 42000000
20:48:21.107       countdown:4   - countdown 42000000
20:48:21.107            main:10  - started threads, will wait
20:48:22.648       countdown:7   - done countdown
20:48:22.727       countdown:7   - done countdown
20:48:22.727            main:14  - finished threads
20:48:22.727            main:16  - done main


CPU times: user 1.6 s, sys: 22.2 ms, total: 1.62 s
Wall time: 1.63 s
